In [31]:
import pandas as pd
import hopsworks
import joblib

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549014
Connected. Call `.close()` to terminate connection gracefully.


In [32]:
mr = project.get_model_registry()
model = mr.get_model("mag_feature", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/mag_detection_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.


In [33]:
feature_view = fs.get_feature_view(name="mag_feature", version=1)

In [34]:
import datetime
from PIL import Image

batch_data = feature_view.get_batch_data()

y_pred = model.predict(batch_data)

y_pred

Finished: Reading data from Hopsworks, using ArrowFlight (1.35s) 


array(['detection', 'detection', 'detection', ..., 'detection',
       'no_detection', 'no_detection'], dtype=object)

In [35]:
batch_data

,x,y,z
0,-3.000000,-978.000000,-299.000000
1,247.000000,-825.000000,-265.000000
2,231.000000,-790.000000,-312.000000
3,443.000000,-451.000000,-282.000000
4,220.000000,-858.000000,-410.000000
...,...,...,...
20405,790.000000,-163.000000,-951.000000
20406,-26.794561,-1250.231269,-1138.426880
20407,907.139231,-194.433694,-761.476655
20408,1257.881354,-762.554906,-2853.992384


In [36]:
latest = y_pred[y_pred.size-1]

In [37]:
mag_fg = fs.get_feature_group(name="mag_feature", version=1)
df = mag_fg.read()
df

Finished: Reading data from Hopsworks, using ArrowFlight (1.14s) 


,x,y,z,mag_cluster
0,-3.000000,-978.000000,-299.000000,detection
1,247.000000,-825.000000,-265.000000,detection
2,231.000000,-790.000000,-312.000000,detection
3,443.000000,-451.000000,-282.000000,detection
4,220.000000,-858.000000,-410.000000,detection
...,...,...,...,...
20405,790.000000,-163.000000,-951.000000,no_detection
20406,-26.794561,-1250.231269,-1138.426880,detection
20407,907.139231,-194.433694,-761.476655,detection
20408,1257.881354,-762.554906,-2853.992384,no_detection


In [38]:
label = df.iloc[-1]["mag_cluster"]
label

'no_detection'

In [39]:
monitor_fg = fs.get_or_create_feature_group(name="mag_feature_prediction_monitoring",
                                  version=1,
                                  primary_key=["datetime"],
                                  description="Magnetic field Prediction/Outcome Monitoring"
                                 )

In [40]:

from datetime import datetime
now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

data = {
    'prediction': [latest],
    'label': [label],
    'datetime': [now],
}
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: mag_feature_prediction_monitoring_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549014/jobs/named/mag_feature_prediction_monitoring_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f2396e793c0>, None)

In [41]:
history_df = monitor_fg.read()
history_df

Finished: Reading data from Hopsworks, using ArrowFlight (0.65s) 


,prediction,label,datetime
0,no_detection,no_detection,"04/19/2024, 11:19:30"
1,no_detection,no_detection,"04/19/2024, 11:26:24"
2,no_detection,no_detection,"04/19/2024, 11:30:45"


In [42]:
df_recent = history_df.tail(5)

In [43]:
from sklearn.metrics import confusion_matrix

predictions = history_df[['prediction']]
labels = history_df[['label']]

results = confusion_matrix(labels, predictions)
print(results)

[[3]]


In [45]:
from matplotlib import pyplot
import seaborn as sns

if results.shape == (2,2):

    df_cm = pd.DataFrame(results, ['True detection', 'False detection'],
                         ['Predicted True', 'Predicted False'])

    cm = sns.heatmap(df_cm, annot=True)

    fig = cm.get_figure()
    fig.savefig("../../pictures/confusion_matrix.png") 
    df_cm
else:
    print("Run the batch inference pipeline more times until you get 3 different magnetic field readings")    

Run the batch inference pipeline more times until you get 3 different magnetic field readings
